# Lab 2: Characteristics of remotely sensed data

**Purpose:** The purpose of this lab is to demonstrate concepts of spatial, spectral, temporal and radiometric resolution.  You will be introduced to image data from several sensors aboard various platforms.  At the completion of the lab, you will be able to understand the difference between remotely sensed datasets based on sensor characteristics and how to choose an appropriate remotely sensed dataset based on these concepts.


In [ ]:
# import ee api and geemap package
import ee
import geemap
from pprint import pprint

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Spatial Resolution

In the present context, spatial resolution means pixel size *(not sensor resolution)*.  For this case, the spatial resolution depends on the projection of the sensor's instantaneous field of view (IFOV) on the ground and how a set of radiometric measurements are resampled into a regular grid.  To see the difference in spatial resolution resulting from different sensors, visualize data at different scales from different sensors.

### MODIS

There are two Moderate Resolution Imaging Spectro-Radiometers ([MODIS](http://modis.gsfc.nasa.gov/)) aboard the [Terra](http://terra.nasa.gov/) and [Aqua](http://aqua.nasa.gov/) satellites.  Different [MODIS bands](http://modis.gsfc.nasa.gov/about/specifications.php) produce data at different spatial resolutions.  For the visible bands, the lowest common resolution is 500 meters (red and NIR are 250 meters). 

Here we will import the '[MYD09GA.006](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/myd09ga_v006) Aqua Surface Reflectance Daily Global 1km and 500m' product.  ([Complete list of MODIS land products](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table)).  Note that Terra MODIS datasets start with '*MOD*' and MODIS Aqua datasets start with '*MYD*').



In [ ]:
# import in the MODIS Aqua surface reflectance product
myd09 = ee.ImageCollection("MODIS/006/MYD09GA")

In [ ]:
# define a region of interest as a point at SFO airport
sfo = ee.Geometry.Point(-122.3774, 37.6194);

In [ ]:
# Get a surface reflectance image from the MODIS MYD09GA collection.
date = "2017-07-01"
modis_image = ee.Image(myd09.filterDate(date).first())

In [ ]:
# Use these MODIS bands for red, green, blue, respectively.
modis_bands = ["sur_refl_b01", "sur_refl_b04", "sur_refl_b03"];

In [ ]:
modis_vis = {"bands": modis_bands, "min": 0, "max": 3300, "gamma":1.3}

In [ ]:
# Display the image
Map = geemap.Map()

Map.centerObject(sfo, 15)

Map.addLayer(modis_image, modis_vis, "MODIS")

Map.addLayerControl()

Map

Note the size of pixels with respect to objects on the ground, these are 250-500m pixels so we cannot resolve fine scale features like an airport. 

In [ ]:
# Get the scale of the data from the first band's projection:
modis_scale = (
    modis_image
    .select('sur_refl_b01')
    .projection()
    .nominalScale()
).getInfo()

print(f"MODIS scale: {modis_scale} meters")

It's also worth noting that these MYD09 data are surface reflectance scaled by 10000 (not top-of-atmosphere (TOA) reflectance), meaning that clever NASA scientists have done a fancy atmospheric correction for you!

If you are interested in reading more on how atmospheric correction works see [this website](https://salsa.umd.edu/6spage.html)

### MSS

The Multispectral Scanner ([MSS](http://landsat.gsfc.nasa.gov/?p=3227)) sensors were flown aboard Landsats 1-5.  MSS data have a spatial resolution of 60 meters.


In [ ]:
# import the Landsat 5 MSS collection
mss = ee.ImageCollection("LANDSAT/LM05/C01/T2")

In [ ]:
# Get the least cloudy image over SFO airport
mss_image = ee.Image(
    mss
    .filterBounds(sfo)
    .sort("CLOUD_COVER") 
    .first()
)


In [ ]:
mss_vis = {"bands": ["B3", "B2", "B1"], "min": 0, "max": 200}

In [ ]:
# Display the MSS image as a color-IR composite.
# Display the image
Map = geemap.Map()

Map.centerObject(sfo, 15)

Map.addLayer(mss_image, mss_vis, "MSS")

Map.addLayerControl()

Map

In [ ]:
# Get the scale of the data from the first band's projection:
mss_scale = (
    mss_image
    .select('B1')
    .projection()
    .nominalScale()
).getInfo()

print(f"MSS scale: {mss_scale} meters")

### TM

The Thematic Mapper ([TM](http://landsat.gsfc.nasa.gov/?p=3229)) was flown aboard Landsat 4-5.  (It was succeeded by the Enhanced Thematic Mapper plus ([ETM+](http://landsat.gsfc.nasa.gov/?p=3225)) aboard Landsat 7 and the Operational Land Imager ([OLI](http://landsat.gsfc.nasa.gov/?p=5447)) / Thermal Infrared Sensor ([TIRS](http://landsat.gsfc.nasa.gov/?p=5474)) sensors aboard Landsat 8.)  TM data have a spatial resolution of 30 meters.


In [ ]:
# load the Landsat 5 TM TOA collection
tm = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")

In [ ]:
# Get the least cloudy image over SFO airport
tm_image = ee.Image(
    tm
    .filterBounds(sfo)
    .sort("CLOUD_COVER") 
    .first()
)

In [ ]:
tm_vis = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.4}

In [ ]:
# Display the TM image as a color-IR composite.
Map = geemap.Map()

Map.centerObject(sfo, 15)

Map.addLayer(tm_image, tm_vis, "TM")

Map.addLayerControl()

Map

For some information about why the TM data is not the same date as the MSS data, see [this page](https://www.usgs.gov/landsat-missions/landsat-5).

In [ ]:
# Get the scale of the data from the first band's projection:
tm_scale = (
    tm_image
    .select('B1')
    .projection()
    .nominalScale()
).getInfo()

print(f"TM scale: {tm_scale} meters")

### NAIP

The National Agriculture Imagery Program ([NAIP](http://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/)) is an effort to acquire imagery over the continental US on a 3-year rotation using airborne sensors.  The imagery have a spatial resolution of 1-2 meters.  

In [ ]:
# load in the NAIP imagery collection
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

In [ ]:
# Get NAIP images for the study period and region of interest.
naip_images = (
    naip
    .filterDate('2012-01-01', '2012-12-31')
    .filterBounds(sfo)
)

In [ ]:
# Mosaic adjacent images into a single image.
naip_mosaic = naip_images.mosaic()

In [ ]:
naip_vis = {"bands": ["N", "R", "G"]}

In [ ]:
# Display the NAIP as a color-IR composite.
Map = geemap.Map()

Map.centerObject(sfo, 15)

Map.addLayer(naip_mosaic, naip_vis, "NAIP")

Map.addLayerControl()

Map

In [ ]:
# Get the scale of the data from the first band's projection:
naip_scale = (
    naip_images
    .first()
    .projection()
    .nominalScale()
).getInfo()

print(f"NAIP scale: {naip_scale} meters")

## Spectral resolution

Spectral resolution refers to the number and width of spectral bands in which the sensor takes measurements.  A sensor that measures radiance in multiple bands is called a multi-spectral sensor, while a sensor with many bands (possibly hundreds) is called a hyper-spectral sensor (these are not hard and fast definitions).  For example, compare the [multi-spectral OLI](http://landsat.gsfc.nasa.gov/?p=5779) aboard Landsat 8 to [Hyperion](https://www.usgs.gov/centers/eros/eo-1-sensors), a hyperspectral sensor aboard the [EO-1 satellite](https://eo1.usgs.gov/).

There is an easy way to check the number of bands in Earth Engine, but no way to get an understanding of the band width or relative spectral response of the bands, where spectral response is a function measured in the laboratory to characterize the detector.  


In [ ]:
# Get the MODIS band names as a List
modis_bands = modis_image.bandNames()

In [ ]:
# Print the list of band names.
bandlist = ',\n'.join(modis_bands.getInfo()) # some string formatting
print(f"MODIS bands: [{bandlist}]")


In [ ]:
# Print the length of the list.
n_bands = modis_bands.length()
print(f'Length of the bands list: {n_bands.getInfo()}')

It's worth noting that only some of those bands contain radiometric data.  Lots of them have other information, like quality control data.  So the band listing isn't necessarily an indicator of spectral resolution, but can inform your investigation of the spectral resolution of the dataset.  Try printing the bands from some of the other sensors to get a sense of spectral resolution.

## Temporal resolution

In this context, we will be discussing temporal resolution as the *revisit time*, or temporal cadence of the image data.  Think of this as the frequency of pixels in a time series at a given location.  


### MODIS

MODIS (either Terra or Aqua) produces imagery at approximately daily cadence.  To see the time series of images at a location, you can print() the ImageCollection, filtered to your area and date range of interest.  For example, to see the MODIS images in 2011:

In [ ]:
# Filter the MODIS mosaics to one year.
modis_series = myd09.filterDate("2011-01-01", "2012-01-01")

In [ ]:
# get the number of images in the filtered collection
n_images = modis_series.size()

print(f"Number of MODIS images: {n_images.getInfo()}")

In [ ]:
# reduce the collection to a single image of valid observations (i.e. not masked)
valid_obs_image = modis_series.reduce(ee.Reducer.count())

# get the number of valid observations over SFO airport for 2011
valid_obs = valid_obs_image.reduceRegion(ee.Reducer.first(), sfo, 500)

pprint(valid_obs.getInfo())

### Landsat

Landsats (5 and later) produce imagery at 16-day cadence.  TM and MSS are on the same satellite (Landsat 5), so it suffices to print the TM series to see the temporal resolution.  Unlike MODIS, data from these sensors is produced on a scene basis, so to see a time series, it's necessary to filter by location in addition to time:


In [ ]:
# Filter the TM collection to one year over SFO airport.
tm_series = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    .filterDate("2018-01-01", "2019-01-01")
    .filterBounds(sfo)
)

In [ ]:
# get the number of images in the filtered collection
n_images = tm_series.size()

print(f"Number of TM Scenes: {n_images.getInfo()}")

To make this into a nicer list of dates, we will `map()` a function over the ImageCollection.  First define a function to get a Date from the metadata of each image, using the system properties:


Turn the ImageCollection into a List and `map()` the function over it:


In [ ]:
def get_date(image):
    # Note that you need to cast the argument
    # get the time of aquisition
    # this is an ee.Date object
    time = ee.Image(image).date();

    # Return the formatted time 
    return time.format("YYYY-MM-dd HH:mm:ss")


In [ ]:
# get a list of Landsat acquisition dates
dates = (
    tm_series
    .toList(n_images) # convert to list
    .map(get_date) # apply the `get_date` function to each element
)

In [ ]:
datelist = ',\n'.join(dates.getInfo()) # some string formatting

print(f"TM acquisition dates over SFO airport: {datelist}")

## Radiometric resolution

Radiometric resolution is determined from the minimum radiance to which the detector is sensitive (Lmin), the maximum radiance at which the sensor saturates (Lmax), and the number of bits used to store the DNs (Q): 

$Radiometric Resolution = (Lmax - Lmin)/2^Q$

It might be possible to dig around in the metadata to find values for Lmin and Lmax, but computing radiometric resolution is generally not necessary unless you're studying phenomena that are distinguished by very subtle changes in radiance.


## Orbits and sensor motion

The image data are collected from moving platforms (satellites or aircraft).  The motion of the platform, together with the imaging geometry of the sensor determines the spatio-temporal resolution of the data.  To get an idea for how these design choices interact to produce the wonderful imagery in Earth Engine, examine the orbit of the Aqua satellite for a selected day in 2013.


In [ ]:
aqua_image = myd09.filterDate('2013-09-01').first()


In [ ]:
sensor_vis = {"bands": 'SensorZenith', "min": 0, "max": 70*100}
solar_vis = {"bands": 'SolarZenith', "min": 0, "max": 70*100}

In [ ]:
aqua_nadir = aqua_image.updateMask(aqua_image.select("SensorZenith").lt(45*100))

In [ ]:
# Display the Aqua viewing geometry
Map = geemap.Map(center=(0,81.04), zoom=3)

Map.addLayer(aqua_nadir, sensor_vis, "Aqua sensor-zenith angle")
Map.addLayer(aqua_image, solar_vis, "Aqua acquisition solar-zenith angle",False)


Map.addLayerControl()

Map

In [ ]:
# Load Landsat ETM+ data directly, filter to one day.
landsat7 = (
    ee.ImageCollection('LANDSAT/LE7')
    .filterDate('2013-09-01', '2013-09-02')
)


In [ ]:
# Display the Aqua viewing geometry
Map = geemap.Map(center=(0,81.04), zoom=3)

Map.addLayer(aqua_image, sensor_vis, "Aqua sensor-zenith angle")
Map.addLayer(landsat7, {"bands": 'B1', "palette": 'blue'}, 'Landsat 7 scenes')

Map.addLayerControl()

Map
